In [1]:
run src/midi_processing.py

In [2]:
from itertools import islice

In [2]:
oh = one_hot_song(corpus[67])
song = converter.parse(corpus[67])

In [2]:
def one_hot_to_midi(one_hot_song, instrument='Saxophone'):
    #events = np.nonzero(one_hot_encoded_song)[0]
    idxs = np.nonzero(one_hot_song)[1]
    this_stream = stream.Stream()
    part = stream.Part()
    inst = instrument.Instrument(instrument)
    temp = tempo.MetronomeMark('animato')
    met = meter.TimeSignature()
    this_stream.append(part)
    part.append(inst)
    part.append(temp)
    part.append(met)
    song_iter = iter(one_hot_song)
    idx = 0
    for val in song_iter:
        if check_for_chord(one_hot_song, idx):
            ch = chord.Chord([get_chord(one_hot_song, idx)], duration=)
            part.append(ch)
            step += get_note_count_from_chord(one_hot_song, idx)
            idx += step+1
            next(islice(song_iter,step,step+1))            
        
        
        
        
        
        
        
        
        if 0 <= val <= 127:
            if check_for_chord(one_hot_song, idx):
                ch = chord.Chord([get_chord(one_hot_song, idx)])
                part.append(ch)
                step += get_note_count_from_chord(one_hot_song, idx)
                idx += step+1
                next(islice(song_iter,step,step+1))
        if 128 <= val <= 255:
            pass
        if 256 <= val <= 356:
            time_in_seconds = (val-255)/100
            quarter_length = temp.secondsToDuration(time_in_seconds).quarterLength
            if idx == 0: #if first element in array represents a time-shift, it is a rest
                r = note.Rest(quarterLength=quarterLength)
                part.append(r)
            elif check_if_rest(one_hot_encoded_song, idx): #check if time shift represents rest
                r = note.Rest(quarterLength=quarterLength)
                part.append(r)
            else: #add time shift to last note
                note = part[idx-2]
                
        if 356 <= val <= 387:
            note = part[idx-1]
            note.volume.velocity = (val-355)*4-1

In [21]:
def get_duration(one_hot_song, time_shift_index):
    dur = 0.0
    idx = time_shift_index 
    dur += ((np.nonzero(one_hot_song)[1][idx])-255)/100
    while 256 <= check_next(one_hot_song, idx) <= 355:
        val = np.nonzero(one_hot_song)[1][idx+1]
        time_in_seconds = (val-255)/100
        dur += time_in_seconds
        idx += 1
    return dur

In [20]:
get_duration(oh, 0)

1.0


3.0

In [9]:
np.nonzero(oh)[1][0]

355

In [18]:
check_next(oh,1)

355